In [1]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
import numpy as np # high dimensional vector computing library.
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import pymongo
import pandas as pd
from pymongo import MongoClient

In [2]:
client = MongoClient()
db = client.stocks

input_data_estadao = db.estadaoNoticias
input_data_folha = db.folhaNoticias

data_estadao = pd.DataFrame(list(input_data_estadao.find()))
data_folha = pd.DataFrame(list(input_data_folha.find()))

data_estadao.head()
data_folha.head()

,_id,conteudo,emissor,idNoticia,repercussao,subFonte,subTitulo,timestamp,titulo,url
0,59398bc80c0adb20e4fe72c7,A exemplo do que faz agora o governo de Michel...,DE SÃO PAULO,FOLHASP-1887182,7,FOLHASP,,1495666440,Dilma também acionou militares contra protesto...,http://www1.folha.uol.com.br/poder/2017/05/188...
1,59398bca0c0adb20e4fe730f,O protesto organizado por centrais sindicais e...,LUCAS VETTORAZZO ANGELA BOLDRINI RUBENS VALENT...,FOLHASP-1887139,71,FOLHASP,,1495658640,"Protesto em Brasília termina com 49 feridos, 8...",http://www1.folha.uol.com.br/poder/2017/05/188...
2,59398bca0c0adb20e4fe7315,"O governador do Distrito Federal, Rodrigo Roll...",LAÍS ALEGRETTI DE BRASÍLIA,FOLHASP-1887134,5,FOLHASP,,1495657560,"'Medida extrema', uso do Exército não teve anu...",http://www1.folha.uol.com.br/poder/2017/05/188...
3,59398bcb0c0adb20e4fe732e,Policiais militares do Distrito Federal utiliz...,ANGELA BOLDRINI LUCAS VETTORAZZO PEDRO LADEIRA...,FOLHASP-1887121,24,FOLHASP,,1495656540,Manifestante é baleado com armamento letal em ...,http://www1.folha.uol.com.br/poder/2017/05/188...
4,59398bcc0c0adb20e4fe7366,Em nota publicada no início da noite desta qua...,DE SÃO PAULO,FOLHASP-1887117,55,FOLHASP,,1495654680,Convocação do Exército remete aos 'anos de chu...,http://www1.folha.uol.com.br/poder/2017/05/188...


In [3]:
#.decode('utf-8')
def tokenize(text):
    try:
        text = text.lower()
        tokens = nltk.word_tokenize(text)
        #tokens = filter(lambda t: not t.startswith('@'), tokens)
        #tokens = filter(lambda t: not t.startswith('#'), tokens)
        #tokens = filter(lambda t: not t.startswith('http'), tokens)
        return tokens
    except:
        return 'NC'

In [4]:
def postprocess(data, n=100):
    #data = data.head(n)
    data['tokens'] = data['conteudo'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

data_estadao = postprocess(data_estadao)
data_folha = postprocess(data_folha)

In [5]:
n_dim = 300

In [6]:
estadao_x_train, estadao_x_test, estadao_y_train, estadao_y_test = train_test_split(np.array(data_estadao.tokens),
                                                    np.array(data_estadao.conteudo), test_size=0.2)

folha_x_train, folha_x_test, folha_y_train, folha_y_test = train_test_split(np.array(data_estadao.tokens),
                                                    np.array(data_estadao.conteudo), test_size=0.2)

In [ ]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

estadao_x_train = labelizeTweets(estadao_x_train, 'TRAIN')
#x_test = labelizeTweets(x_test, 'TEST')

folha_x_train = labelizeTweets(folha_x_train, 'TRAIN')
#x_test = labelizeTweets(x_test, 'TEST')


In [ ]:
estadao_word2vec = Word2Vec(size=n_dim, min_count=10)
estadao_word2vec.build_vocab([x.words for x in tqdm(estadao_x_train)])
estadao_word2vec.train([x.words for x in tqdm(estadao_x_train)], total_examples=estadao_word2vec.corpus_count, epochs=estadao_word2vec.iter)

folha_word2vec = Word2Vec(size=n_dim, min_count=10)
folha_word2vec.build_vocab([x.words for x in tqdm(folha_x_train)])
folha_word2vec.train([x.words for x in tqdm(folha_x_train)], total_examples=folha_word2vec.corpus_count, epochs=folha_word2vec.iter)


In [ ]:
estadao_word2vec.most_similar("gay")

In [40]:
estadao_word2vec.most_similar("homossexual")

[(u'gay', 0.5766882300376892),
 (u'aborto', 0.5760660767555237),
 (u'casamento', 0.5567106604576111),
 (u'homof\xf3bico', 0.5503756403923035),
 (u'racista', 0.5388685464859009),
 (u'preconceito', 0.5338813662528992),
 (u'homossexualidade', 0.4838162362575531),
 (u'ignorante', 0.4747411608695984),
 (u'homossexuais', 0.4683874845504761),
 (u'religiosa', 0.4627786874771118)]

In [41]:
folha_word2vec.most_similar("gay")

[(u'kit', 0.6854844093322754),
 (u'anti-homofobia', 0.6581751108169556),
 (u'homossexual', 0.6365687847137451),
 (u'maconha', 0.6100272536277771),
 (u'aborto', 0.6005829572677612),
 (u'homofobia', 0.5945218801498413),
 (u'lgbt', 0.5925257205963135),
 (u'legaliza\xe7\xe3o', 0.5647957921028137),
 (u'descriminaliza\xe7\xe3o', 0.5590815544128418),
 (u'feminista', 0.5416308045387268)]

In [42]:
folha_word2vec.most_similar("homossexual")

[(u'gay', 0.6365687847137451),
 (u'preconceito', 0.5623294115066528),
 (u'aborto', 0.5586243867874146),
 (u'casamento', 0.53482586145401),
 (u'homof\xf3bico', 0.5249391794204712),
 (u'racista', 0.5217532515525818),
 (u'homossexuais', 0.5212612152099609),
 (u'racismo', 0.5107216835021973),
 (u'discriminat\xf3rio', 0.5000503659248352),
 (u'maconha', 0.4920646846294403)]

In [35]:
estadao_word2vec.most_similar("branco")

[(u'janeiro/rj', 0.5468244552612305),
 (u'paraty', 0.5260704755783081),
 (u'ipanema', 0.5153312683105469),
 (u'areia', 0.5045171976089478),
 (u'niter\xf3i', 0.4898533225059509),
 (u'doce', 0.4682314991950989),
 (u'florian\xf3polis', 0.4555416703224182),
 (u'cristo', 0.4485395848751068),
 (u'nulo', 0.43999797105789185),
 (u'preto', 0.43848860263824463)]

In [34]:
estadao_word2vec.most_similar("branco")

[(u'janeiro/rj', 0.5468244552612305),
 (u'paraty', 0.5260704755783081),
 (u'ipanema', 0.5153312683105469),
 (u'areia', 0.5045171976089478),
 (u'niter\xf3i', 0.4898533225059509),
 (u'doce', 0.4682314991950989),
 (u'florian\xf3polis', 0.4555416703224182),
 (u'cristo', 0.4485395848751068),
 (u'nulo', 0.43999797105789185),
 (u'preto', 0.43848860263824463)]

In [32]:
folha_word2vec.most_similar("negro")

[(u'oper\xe1rio', 0.6695019006729126),
 (u'revolucion\xe1rio', 0.5380944013595581),
 (u'jogador', 0.5246587991714478),
 (u'comunista', 0.5224142074584961),
 (u'ditador', 0.5202620625495911),
 (u'comunismo', 0.5068507194519043),
 (u'guerrilheiro', 0.504123330116272),
 (u'sindicalismo', 0.49361711740493774),
 (u'judeu', 0.4928639531135559),
 (u'clandestino', 0.4919804334640503)]

In [33]:
folha_word2vec.most_similar("branco")

[(u'doce', 0.5055692791938782),
 (u'copacabana', 0.49736350774765015),
 (u'areia', 0.492209792137146),
 (u'ipanema', 0.4825515151023865),
 (u'bonito', 0.47059184312820435),
 (u'florian\xf3polis', 0.4678967595100403),
 (u'bangu', 0.45894283056259155),
 (u'bel\xe9m', 0.457760214805603),
 (u'niter\xf3i', 0.44850626587867737),
 (u'xingu', 0.44001996517181396)]

In [31]:
folha_word2vec.most_similar("mulher")

[(u'm\xe3e', 0.6767886281013489),
 (u'esposa', 0.6735522150993347),
 (u'filha', 0.6537207365036011),
 (u'amiga', 0.6239906549453735),
 (u'primeira-dama', 0.6065464019775391),
 (u'av\xf3', 0.6048905849456787),
 (u'irm\xe3', 0.6046347618103027),
 (u'dama', 0.5851244330406189),
 (u'companheira', 0.5654486417770386),
 (u'namorada', 0.5650165677070618)]

In [30]:
folha_word2vec.most_similar("homem")

[(u'estadista', 0.580707848072052),
 (u'abra\xe7o', 0.5796175003051758),
 (u'rapaz', 0.5783818960189819),
 (u'pensador', 0.5630857348442078),
 (u'amigo', 0.5576744675636292),
 (u'administrador', 0.5552319288253784),
 (u'soldado', 0.5533406734466553),
 (u'menino', 0.5510196089744568),
 (u'funcion\xe1rio', 0.5487581491470337),
 (u'jogador', 0.5408045649528503)]

In [29]:
estadao_word2vec.most_similar("mulher")

[(u'esposa', 0.6920737028121948),
 (u'm\xe3e', 0.6857017278671265),
 (u'filha', 0.6713874340057373),
 (u'irm\xe3', 0.633479654788971),
 (u'namorada', 0.614106297492981),
 (u'amiga', 0.604663610458374),
 (u'av\xf3', 0.5999443531036377),
 (u'primeira-dama', 0.5966878533363342),
 (u'dama', 0.5859726071357727),
 (u'ex-mulher', 0.5759395360946655)]

In [28]:
estadao_word2vec.most_similar("homem")

[(u'estadista', 0.6457992792129517),
 (u'menino', 0.5838231444358826),
 (u'lutador', 0.5796926021575928),
 (u'rapaz', 0.5656730532646179),
 (u'abra\xe7o', 0.560944676399231),
 (u'amigo', 0.5564925074577332),
 (u'soldado', 0.551363468170166),
 (u'funcion\xe1rio', 0.5411416292190552),
 (u'g\xeanio', 0.5387078523635864),
 (u'gestor', 0.5334969758987427)]

In [26]:
folha_word2vec.most_similar("ela")

[(u'ele', 0.7416077852249146),
 (u'ningu\xe9m', 0.539893627166748),
 (u'voc\xea', 0.5280378460884094),
 (u'algu\xe9m', 0.5206277966499329),
 (u'marina', 0.5202836394309998),
 (u'gente', 0.5014085173606873),
 (u'quem', 0.4936975836753845),
 (u'dela', 0.46034401655197144),
 (u'sra', 0.4562416672706604),
 (u'aquilo', 0.45602965354919434)]

In [27]:
folha_word2vec.most_similar("ele")

[(u'ela', 0.7416078448295593),
 (u'algu\xe9m', 0.5628678202629089),
 (u'ningu\xe9m', 0.5409524440765381),
 (u'quem', 0.5180811285972595),
 (u'voc\xea', 0.48269379138946533),
 (u'peemedebista', 0.4629306197166443),
 (u'gente', 0.45480236411094666),
 (u'mim', 0.4464748501777649),
 (u'senhor', 0.4436030387878418),
 (u'aquilo', 0.4421740174293518)]

In [13]:
# importing bokeh library for interactive dataviz
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

# defining the chart
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="A map of 10000 word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

# getting a list of word vectors. limit to 10000. each is of 200 dimensions
word_vectors = [tweet_w2v[w] for w in tweet_w2v.wv.vocab.keys()[:5000]]

# dimensionality reduction. converting the vectors to 2d vectors
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

# putting everything in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = tweet_w2v.wv.vocab.keys()[:5000]

# plotting. the corresponding word appears when you hover on the data point.
plot_tfidf.scatter(x='x', y='y', source=tsne_df)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_tfidf)

Loading BokehJS ...

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 5000
[t-SNE] Computed conditional probabilities for sample 2000 / 5000
[t-SNE] Computed conditional probabilities for sample 3000 / 5000
[t-SNE] Computed conditional probabilities for sample 4000 / 5000
[t-SNE] Computed conditional probabilities for sample 5000 / 5000
[t-SNE] Mean sigma: 0.171422
[t-SNE] KL divergence after 100 iterations with early exaggeration: 1.209544
[t-SNE] Error after 325 iterations: 1.209544


ValueError: expected an instance of type DataSource, got               x          y        words
0     -9.082752  -5.392836      raining
1     -4.783958   4.620254       yellow
2     -3.205264  -6.151708         four
3     -9.916620   3.989376        woods
4     -8.508235  -1.975228      spiders
5     -3.356022   4.113865     friend's
6     -2.671075   6.280012      hanging
7     -8.578102   2.872470  electricity
8     -5.045470  -0.142866         lord
9    -10.920730  -0.617917      shaving
10    -9.465813   0.125165       callin
11    -4.542277   2.504884    screaming
12     0.952688   5.277322        prize
13    -0.318385  -6.135655    wednesday
14    -8.672418  -1.018768         oooo
15    -5.236122  -0.606604         oooh
16    -9.506723   0.335415    commented
17    -7.249676  -5.185357        tired
18    -4.435387   5.190530        bacon
19    -9.286023   3.107189        tires
20    -3.610883  -5.043841       second
21    -7.537213   3.401917       errors
22    -6.617863  -2.030